In [13]:
library(data.table)

In [14]:
prices = fread('aws_instance_pricing/pricing1.csv')
prices[, cpu := as.numeric(`vCPU*`)]
prices[1:3]

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”


Instance,vCPU*,CPU Credits/hour,Mem (GiB),Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu
<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
t3.nano,2,6,0.5,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2
t3.micro,2,12,1.0,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2
t3.small,2,24,2.0,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2


In [15]:
spot = fread('aws_instance_pricing/spot_pricing_20211129.csv')
spot[, cost2 := as.numeric(gsub('\\$','',gsub(' .*','',Cost)))]
# spot[, cost2 := gsub('\\$','',gsub(' .*','',Cost))]
spot[1:3]

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”


Instance,Cost,Windows,cost2
<chr>,<chr>,<chr>,<dbl>
a1.medium,$0.0049 per Hour,N/A*,0.0049
a1.large,$0.0098 per Hour,N/A*,0.0098
a1.xlarge,$0.0197 per Hour,N/A*,0.0197


In [16]:
m = merge(spot, prices, by='Instance')
m[1:3]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,Mem (GiB),Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
c5.12xlarge,$0.4854 per Hour,$2.6641 per Hour,0.4854,48,NA,96,EBS-Only,12,9500,Intel Xeon Scalable,48
c5.18xlarge,$0.7032 per Hour,$3.9961 per Hour,0.7032,72,NA,144,EBS-Only,25,19000,Intel Xeon Scalable,72
c5.24xlarge,$0.9234 per Hour,$5.3282 per Hour,0.9234,96,NA,192,EBS-Only,25,19000,Intel Xeon Scalable,96


In [17]:
m[, cost_per_cpu := cost2/cpu]
m[order(cost_per_cpu)][1:5]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,Mem (GiB),Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu,cost_per_cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
t3.nano,$0.0016 per Hour,$0.0062 per Hour,0.0016,2,6,0.5,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2,0.00080
t3a.nano,$0.0016 per Hour,$0.006 per Hour,0.0016,2,6,0.5,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00080
t3a.micro,$0.003 per Hour,$0.012 per Hour,0.0030,2,12,1.0,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00150
t3.micro,$0.0031 per Hour,$0.0123 per Hour,0.0031,2,12,1.0,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2,0.00155
t3a.small,$0.0056 per Hour,$0.024 per Hour,0.0056,2,24,2.0,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00280


In [18]:
setnames(m, 'Mem (GiB)', 'RAM')
m[RAM/cpu >= 2][order(cost_per_cpu)][1:5]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,RAM,Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu,cost_per_cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
t3a.medium,$0.0113 per Hour,$0.0297 per Hour,0.0113,2,24,4,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.00565
t3.medium,$0.0125 per Hour,$0.0309 per Hour,0.0125,2,24,4,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2,0.00625
t2.small,$0.0069 per Hour,$0.0159 per Hour,0.0069,1,12,2,EBS-Only,Low to Moderate,,3.3 GHz Intel Scalable Processor,1,0.00690
t2.medium,$0.0139 per Hour,$0.0319 per Hour,0.0139,2,24,4,EBS-Only,Low to Moderate,,3.3 GHz Intel Scalable Processor,2,0.00695
c5.4xlarge,$0.152 per Hour,$0.888 per Hour,0.1520,16,NA,32,EBS-Only,Up to 10,4750,Intel Xeon Scalable,16,0.00950


In [22]:
m[RAM/cpu >= 2][order(cost_per_cpu)]

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,RAM,Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu,cost_per_cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
t3a.medium,$0.0113 per Hour,$0.0297 per Hour,0.0113,2,24,4,EBS-Only,Up to 5,,AMD EPYC 7000 series processors with an all core turbo clock speed of 2.5 GHz,2,0.005650000
t3.medium,$0.0125 per Hour,$0.0309 per Hour,0.0125,2,24,4,EBS-Only,Up to 5,,"Up to 3.1 GHz Intel Xeon Platinum Processor, Intel AVX†, Intel AVX2†, Intel Turbo",2,0.006250000
t2.small,$0.0069 per Hour,$0.0159 per Hour,0.0069,1,12,2,EBS-Only,Low to Moderate,,3.3 GHz Intel Scalable Processor,1,0.006900000
t2.medium,$0.0139 per Hour,$0.0319 per Hour,0.0139,2,24,4,EBS-Only,Low to Moderate,,3.3 GHz Intel Scalable Processor,2,0.006950000
c5.4xlarge,$0.152 per Hour,$0.888 per Hour,0.1520,16,NA,32,EBS-Only,Up to 10,4750,Intel Xeon Scalable,16,0.009500000
c5d.large,$0.019 per Hour,$0.111 per Hour,0.0190,2,NA,4,1 x 50 NVMe SSD,Up to 10,"Up to 4,750",Intel Xeon Scalable,2,0.009500000
c5d.xlarge,$0.038 per Hour,$0.222 per Hour,0.0380,4,NA,8,1 x 100 NVMe SSD,Up to 10,"Up to 4,750",Intel Xeon Scalable,4,0.009500000
c5d.18xlarge,$0.6841 per Hour,$3.9961 per Hour,0.6841,72,NA,144,2 x 900 NVMe SSD,25,19000,Intel Xeon Scalable,72,0.009501389
c5.large,$0.0191 per Hour,$0.111 per Hour,0.0191,2,NA,4,EBS-Only,Up to 10,"Up to 4,750",Intel Xeon Scalable,2,0.009550000


In [25]:
m[Instance=='c5d.18xlarge' | Instance=='c5d.24xlarge']

Instance,Cost,Windows,cost2,vCPU*,CPU Credits/hour,RAM,Storage,Network Performance (Gbps)***,EBS Bandwidth (Gbps),Processor,cpu,cost_per_cpu
<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
c5d.18xlarge,$0.6841 per Hour,$3.9961 per Hour,0.6841,72,NA,144,2 x 900 NVMe SSD,25,19000,Intel Xeon Scalable,72,0.009501389
c5d.24xlarge,$1.0176 per Hour,$5.3282 per Hour,1.0176,96,NA,192,4 x 900 NVMe SSD,25,19000,Intel Xeon Scalable,96,0.010600000


In [23]:
50*72*2

[1] 7200